In [64]:
# import python packages

import pandas as pd
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras import applications
from keras.models import Sequential
from keras.layers import Flatten, Dense
from sklearn.naive_bayes import MultinomialNB
import os
from sklearn import metrics


# Create the dataset from scratch

Crawl the github repos to find the code files/snippets written in different languages. Easiest way i used was to clone the relevant github repos and used glob.glob with the file extensions to get the path names of all the files and saved the files in different folders  using shutil copy command. 

In [2]:
# Create separate folders for each file extension type to store the corresponding files
os.makedirs("D:\\dataset\\test\\java")
os.makedirs("D:\\dataset\\test\\python")
os.makedirs("D:\\dataset\\test\\javascript")
os.makedirs("D:\\dataset\\test\\c")

In [ ]:
# Get list of all .java files from all the subdirectories of your cloned repo and save them

import os, glob
import shutil

destination = "D:\\dataset\\test\\java"

for filename in glob.glob('C:\\Users\\Yogita\\Downloads\\guava-master\\guava-master\\**\\*.java', recursive = True):
        shutil.copy(filename, destination)
      

In [ ]:
# Get list of all .python files from all the subdirectories of your cloned repo and save them

destination = "D:\\dataset\\test\\python"

for filename in glob.glob('C:\\Users\\Downloads\\repo_name\\**\\*.py', recursive = True):
        shutil.copy(filename, destination)

In [ ]:
# Get list of all .c files from all the subdirectories of your repo

destination = "D:\\dataset\\test\\c"

for filename in glob.glob('C:\\Users\\Downloads\\repo_name\\**\\*.c', recursive = True):
        shutil.copy(filename, destination)

In [ ]:
# Get list of all .js files from all the subdirectories of your repo

destination = "D:\\dataset\\test\\c"

for filename in glob.glob('C:\\Users\\Downloads\\repo_name\\**\\*.js', recursive = True):
        shutil.copy(filename, destination)

## Loading the data and converting into dataframe for further processing

'load_files' function of 'sklearn.datasets' package loads the files from the subfolders (created above for each type of file), the real data lies in the filenames and the subfolder names are treated as categories. This function returns a dictionary object which has attributes like filenames (of the files holding data), the target or classification labels, label names and description of the dataset.

In [81]:
# load the data files

rawdata = skds.load_files("D:\dataset",load_content=False)
rawdata
#print(rawdata.target_names, rawdata.target)

{'filenames': array(['D:\\dataset\\java\\ConverterTest.java',
        'D:\\dataset\\python\\inspect.py',
        'D:\\dataset\\java\\LinkedHashMultiset_CustomFieldSerializer.java',
        ..., 'D:\\dataset\\java\\EscapersTest_gwt.java',
        'D:\\dataset\\java\\ForwardingBlockingDeque.java',
        'D:\\dataset\\java\\SortedIterablesTest.java'], dtype='<U75'),
 'target_names': ['c', 'java', 'javascript', 'python'],
 'target': array([1, 3, 1, ..., 1, 1, 1]),
 'DESCR': None}

In [4]:
# read data from the files and append to make a list

files = []
i=0
for file in rawdata.filenames:
    files.append((file,rawdata.target_names[rawdata.target[i]],rawdata.target[i],Path(file).read_text(encoding="utf8")))
    i += 1

# Convert the Filename, label and data available as list into a dataframe

data = pd.DataFrame.from_records(files, columns = ('filename', 'language', 'label','text'))
data.head()


,filename,language,label,text
0,D:\dataset\java\ConverterTest.java,java,1,/*\n * Copyright (C) 2008 The Guava Authors\n ...
1,D:\dataset\python\inspect.py,python,3,"""""""Get useful information from live Python obj..."
2,D:\dataset\java\LinkedHashMultiset_CustomField...,java,1,/*\n * Copyright (C) 2009 The Guava Authors\n ...
3,D:\dataset\java\SetFeature.java,java,1,/*\n * Copyright (C) 2008 The Guava Authors\n ...
4,D:\dataset\java\ImmutableMapTest_gwt.java,java,1,/*\n * Copyright (C) 2008 The Guava Authors\n ...


## Create train and test datasets

In [5]:

random.seed(23)

filename_train, filename_test, y_train, y_test, label_train, label_test, data_train, data_test = train_test_split(
    data.filename, data.language, data.label, data.text, test_size=0.2)

data_train.head()


1556    /*\n * Copyright (C) 2005 The Guava Authors\n ...
2045    /*\n * Copyright (C) 2009 The Guava Authors\n ...
1189    /*\n * Copyright (C) 2009 The Guava Authors\n ...
1008    /*\n * Copyright (C) 2009 The Guava Authors\n ...
431     /*\n * Copyright (C) 2008 The Guava Authors\n ...
Name: text, dtype: object

In [6]:
print(len(data_train), len(data_test), len(y_train), len(y_test), len(filename_train), len(filename_test))
#print(y_train)


2055 514 2055 514 2055 514


# Feature Extraction

Having created the train and test datasets, let us apply machine learning to identify the type of language a file is written in.
To do so, the text has to be converted to numeric tensors, this is called Vectorizing the text. The process invloves breaking the text down to either characters or words or n-grams called tokens called Tokenisation, and then associating a numeric vector with each token. 

I used two techniques to convert the text into vectors - One hot encoding and Word embedding using Keras. The output of One hot encoding is fed to the Naive Bayes classifier, and the output of word embedding layer is fed to the deep neural networks.


## Word level One-hot encoding using Keras 
This uses keras utilities to convert the raw text into sparse vector representation which is fed to Naive bayes classifier

In [60]:
# 1. Word level One-hot encoding using Keras  - 

max_features = 10000

# Creates a tokenizer to take into account top 10,000 most common words
tokenizer = Tokenizer(num_words=max_features  )

# fit on the tokens to build the word index
tokenizer.fit_on_texts(data_train)

# converts strings into word indices
sequences = tokenizer.texts_to_sequences(data_train)

# Convert the words into vectors
X_train = tokenizer.texts_to_matrix(data_train, mode='tfidf')
# Apply the same transformation on the test data
X_test = tokenizer.texts_to_matrix(data_test, mode='tfidf')

# word index
dictionary = tokenizer.word_index
print('Found %s unique tokens.' % len(dictionary) )
#print('Found %s unique tokens.' % len(dictionary),dictionary )

Found 71949 unique tokens.


In [61]:
# train the Naive Bayes classifier model

nbmodel = MultinomialNB().fit(X_train, y_train)

In [94]:
# Evaluation of the performance on the test set

predicted = nbmodel.predict(X_test)
print("Accuracy:", np.mean(predicted == y_test))

metrics.confusion_matrix(y_test, predicted)
prob = nbmodel.predict_proba(X_test)

#print(metrics.classification_report(y_test, predicted, target_names=y_test))

Accuracy: 0.9980544747081712


In [93]:
# Output - Test data set with actual and predicted labels

for i in range(len(x_test)):
    print(filename_test.iloc[i] + ' => ' + 'Actual label:' + y_test.iloc[i] +', ' + "Predicted label: " + predicted[i])
    print(rawdata.target_names, prob[i].round())
    

D:\dataset\java\CollectionRetainAllTester.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\FloatsTest.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\UncaughtExceptionHandlers.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\TestSortedMapGenerator.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\ArrayBasedUnicodeEscaperTest_gwt.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\javascript\test-cli-eval-event.js => Actual label:javascript, Predicted label: javascript
['c', 'java', 'javascript', 'python'] [0. 0. 1. 0.]
D:\dataset\java\EventBus.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\p

D:\dataset\java\StatsAccumulator.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\MinimalIterableTest.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\SetOperationsTest_gwt.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\c\xdp_redirect_map_user.c => Actual label:c, Predicted label: c
['c', 'java', 'javascript', 'python'] [1. 0. 0. 0.]
D:\dataset\javascript\test-cluster-invalid-message.js => Actual label:javascript, Predicted label: javascript
['c', 'java', 'javascript', 'python'] [0. 0. 1. 0.]
D:\dataset\python\install_egg_info.py => Actual label:python, Predicted label: python
['c', 'java', 'javascript', 'python'] [0. 0. 0. 1.]
D:\dataset\python\webbrowser.py => Actual label:python, Predicted label: python
['c', 'java', 'javascript', 'python'] [0. 0. 0. 1.]
D:\dataset\java\CacheBuilderSpe

['c', 'java', 'javascript', 'python'] [0. 0. 0. 1.]
D:\dataset\java\ListTestSuiteBuilder.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\javascript\test-buffer-slice.js => Actual label:javascript, Predicted label: javascript
['c', 'java', 'javascript', 'python'] [0. 0. 1. 0.]
D:\dataset\javascript\test-atomics-notify.js => Actual label:javascript, Predicted label: javascript
['c', 'java', 'javascript', 'python'] [0. 0. 1. 0.]
D:\dataset\c\map.c => Actual label:c, Predicted label: c
['c', 'java', 'javascript', 'python'] [1. 0. 0. 0.]
D:\dataset\java\StandardRowSortedTable.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\PairedStats.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python'] [0. 1. 0. 0.]
D:\dataset\java\TreeMultimapNaturalTest_gwt.java => Actual label:java, Predicted label: java
['c', 'java', 'javascript', 'python

# Predict the language of the (unseen) test code file

In [79]:
# Predict on new data
test_files_path = "D:\\dataset - extra files\\test data"
test_data = []    
for filename in os.listdir(test_files_path):
    path = os.path.join(test_files_path, filename)
    with open(path, encoding="utf8") as f:
        read_data = f.read().split()
        test_data.append(read_data)
test_data1 = pd.Series(test_data)
#print(test_data1.head())

t_data = tokenizer.texts_to_matrix(test_data1, mode='tfidf')


In [95]:
i=0
for t in t_data:
    prec = nbmodel.predict(np.array([t]))
    print("File:", os.listdir(test_files_path)[i] ,'-> Predicted label: ' + prec[0])
    i+=1

File: AbstractAbstractFutureTest.java -> Predicted label: java
File: AbstractBiMap.java -> Predicted label: java
File: fmc-private.h -> Predicted label: c
File: sock_example.h -> Predicted label: c
File: test-crypto-hash-stream-pipe.js -> Predicted label: javascript
File: test-crypto-hash.js -> Predicted label: javascript
File: test-crypto-hmac.js -> Predicted label: javascript
File: test-crypto-keygen.js -> Predicted label: javascript
File: test-fs-error-messages.js -> Predicted label: javascript
File: test-readline-interface.js -> Predicted label: python
File: test-util-inspect.js -> Predicted label: javascript
File: xdpsock.h -> Predicted label: c
File: xdp_tx_iptunnel_common.h -> Predicted label: c
File: __init__.py -> Predicted label: python


## using Keras word embedding

In [26]:
# Feature Extraction

max_features = 10000

# Creates a tokenizer to take into account top 10,000 most common words
tokenizer = Tokenizer(num_words=max_features  )

# fit on the tokens to build the word index
tokenizer.fit_on_texts(data_train)

# converts strings into word indices
sequences = tokenizer.texts_to_sequences(data_train)

# Converst the words into vectors
x_train = tokenizer.texts_to_matrix(data_train, mode='tfidf')
x_test = tokenizer.texts_to_matrix(data_test, mode='tfidf')

x_train = pad_sequences(x_train, 500)
x_test = pad_sequences(x_test, 500)

#dictionary = tokenizer.word_index
#print('Found %s unique tokens.' % len(dictionary) )
#print('Found %s unique tokens.' % len(dictionary),dictionary )


In [8]:
# categorise Target Classes

from keras.utils.np_utils import to_categorical

Y_train = to_categorical(label_train)
Y_test = to_categorical(label_test)

In [45]:
# Keras Model building and fitting

model = Sequential()

model.add(Embedding(max_features, 100, input_length=500))
model.add(Flatten())
model.add(Dense(512, activation='sigmoid', input_shape = (10000,)))
#model.add(Dropout(0.2))
#model.add(Dense(512, activation='sigmoid'))
#model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  
history = model.fit(x_train, Y_train,
                     epochs=15,
                     batch_size=32, verbose =1, validation_split=0.1)
  


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 100)          1000000   
_________________________________________________________________
flatten_9 (Flatten)          (None, 50000)             0         
_________________________________________________________________
dense_41 (Dense)             (None, 512)               25600512  
_________________________________________________________________
dense_42 (Dense)             (None, 4)                 2052      
Total params: 26,602,564
Trainable params: 26,602,564
Non-trainable params: 0
_________________________________________________________________
Train on 1849 samples, validate on 206 samples
Epoch 1/15
1849/1849 [==============================] - 42s 23ms/step - loss: 1.2702 - acc: 0.6252 - val_loss: 1.1529 - val_acc: 0.5777
Epoch 2/15
1849/1849 [==============================] - 38s 21ms/step - loss: 1

In [46]:
# Model evaluation

score = model.evaluate(x_test, Y_test, batch_size=32, verbose=0)
 
print('loss: ' , score[0], 'Test accuracy:', score[1])


loss:  0.7527100887975804 Test accuracy: 0.7704280155642024


In [91]:
# Prediction on the test dataset
prediction = model.predict(x_test)
for i in range(len(x_test)):
    
    print(filename_test.iloc[i] + ' => '  , 'Actual label:' , Y_test[i],  ', ' , "Predicted label: " , prediction[i].round(2))
    

D:\dataset\java\CollectionRetainAllTester.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0. 1. 0. 0.]
D:\dataset\java\FloatsTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.01 0.98 0.01 0.01]
D:\dataset\java\UncaughtExceptionHandlers.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.02 0.88 0.03 0.08]
D:\dataset\java\TestSortedMapGenerator.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.   0.99 0.   0.  ]
D:\dataset\java\ArrayBasedUnicodeEscaperTest_gwt.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\javascript\test-cli-eval-event.js =>  Actual label: [0. 0. 1. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\EventBus.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0. 1. 0. 0.]
D:\dataset\java\package-info.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\AbstractPackageSanityTests.java =>  Actual label: [0. 1. 0. 0.] ,  

D:\dataset\java\MapInterfaceTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.01 0.96 0.01 0.01]
D:\dataset\python\_py_abc.py =>  Actual label: [0. 0. 0. 1.] ,  Predicted label:  [0.42 0.16 0.01 0.41]
D:\dataset\java\ImmutableSortedMapTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.01 0.96 0.02 0.01]
D:\dataset\java\MultimapPutIterableTester.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\javascript\test-child-process-exec-stdout-stderr-data-string.js =>  Actual label: [0. 0. 1. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\python\ntpath.py =>  Actual label: [0. 0. 0. 1.] ,  Predicted label:  [0. 0. 0. 1.]
D:\dataset\java\CaseFormat.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.08 0.22 0.   0.69]
D:\dataset\java\ConcurrentMap.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\FuturesTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted la

D:\dataset\java\ConfigurableSimpleDirectedNetworkTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.01 0.98 0.01 0.  ]
D:\dataset\java\AbstractIdleServiceTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\MultimapPutAllMultimapTester.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\LinkedHashMultisetTest_gwt.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\MultiReader.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\SetsTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\javascript\test-buffer-writefloat.js =>  Actual label: [0. 0. 1. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\javascript\test-arm-math-illegal-instruction.js =>  Actual label: [0. 0. 1. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\python\mimetypes.

D:\dataset\java\AbstractIndexedListIterator.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0. 1. 0. 0.]
D:\dataset\javascript\test-child-process-exec-env.js =>  Actual label: [0. 0. 1. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\ExecutionList.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.   0.89 0.   0.1 ]
D:\dataset\java\GeneralRange.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.02 0.94 0.03 0.02]
D:\dataset\java\UnmodifiableCollectionTests.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\MultisetReadsTester.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\ConcurrentHashMultiset.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\CycleDetectingLockFactoryBenchmark.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\javascript\test-buffer-readdouble.js

D:\dataset\java\NetworkEquivalenceTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\AbstractInvocationHandlerTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\AbstractByteHasherTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\CharsetsTest.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.   0.99 0.   0.  ]
D:\dataset\python\abc.py =>  Actual label: [0. 0. 0. 1.] ,  Predicted label:  [0. 0. 0. 1.]
D:\dataset\c\tracex3_kern.c =>  Actual label: [1. 0. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\c\da8xx-mstpri.c =>  Actual label: [1. 0. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\c\h4_recv.h =>  Actual label: [1. 0. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\dataset\java\MapMakerComparisonBenchmark.java =>  Actual label: [0. 1. 0. 0.] ,  Predicted label:  [0.11 0.67 0.14 0.08]
D:\data

# Prediction

In [56]:
test_files_path = "D:\\dataset - extra files\\test data"
test_data = []    
for filename in os.listdir(test_files_path):
    path = os.path.join(test_files_path, filename)
    with open(path, encoding="utf8") as f:
        read_data = f.read().split()
        test_data.append(read_data)
test_data1 = pd.Series(test_data)
#print(test_data1.head())

t_data = tokenizer.texts_to_matrix(test_data1, mode='tfidf')
test_dat = pad_sequences(t_data, 500)
i=0
for t in test_dat:
    predict = model.predict(np.array([t]))
    print("File:", os.listdir(test_files_path)[i] ,'                 -> Predicted label: ' , predict[0].round(2))
    i+=1

File: AbstractAbstractFutureTest.java                  -> Predicted label:  [0.01 0.97 0.01 0.01]
File: AbstractBiMap.java                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: fmc-private.h                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: sock_example.h                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: test-crypto-hash-stream-pipe.js                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: test-crypto-hash.js                  -> Predicted label:  [0.08 0.45 0.15 0.31]
File: test-crypto-hmac.js                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: test-crypto-keygen.js                  -> Predicted label:  [0.1  0.4  0.44 0.06]
File: test-fs-error-messages.js                  -> Predicted label:  [0.01 0.02 0.01 0.96]
File: test-readline-interface.js                  -> Predicted label:  [0.11 0.67 0.14 0.08]
File: test-util-inspect.js                  -> Predicted label:  [0.02 0.04 0.93 0.01]
File: xdpsock.h           

The output gives the probability of a file belonging to each of the four language type, for example, the first file "AbstractAbstractFutureTest.java" is a java file with 97% probability and so on.

# Conclusion
The Naive Bayes classifer gave an accuracy of 99.81% while the neural network gave an accuracy of 77.04%. The accuracy of the neural network can be improved further by changing the configuration of the sequential model provided by the Keras Sequential Model API.